# San Francisco Rental Prices Dashboard

In [8]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv


import warnings
warnings.filterwarnings('ignore')

In [3]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [9]:
# Import the necessary CSVs to Pandas DataFrames

file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path_2 = Path("Data/neighborhoods_coordinates.csv")
coordinate_data = pd.read_csv(file_path)

- - -

## Panel Visualizations

In [5]:
# Define Panel Visualization Functions

def housing_units_per_year():
    """Housing Units Per Year."""

    housing_units_per_year = sfo_data["housing_units"].groupby("year").mean()
    housing_units_per_year = pd.DataFrame(housing_units_per_year) 
    housing_units_per_year = housing_units_per_year.hvplot.bar(label="Housing Units in San Fransisco from 2010 to 2016",
                                      xlabel = "Year", 
                                      ylabel = "Housing Units",
                                      ylim = (370000, 387500),
                                      height = 300,
                                      width = 400,
                                     ).opts(yformatter="%.0f")
    return housing_units_per_year

def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    
    average_gross_rent = sfo_data["gross_rent"].groupby(["year"]).mean()
    average_gross_rent = pd.DataFrame(average_gross_rent)
    average_gross_rent = average_gross_rent.gross_rent.hvplot.line(
        height = 300, 
        width = 400,
        xlabel = "Year", 
        line_color = "Orange", 
        line_width = 2, 
        ylabel = "Average Gross Rent", 
        ylim = (1000, 4500), 
        title = "Average Rent by Year")
    return average_gross_rent


def average_sales_price():
    """Average Sales Price Per Year."""
    
    average_sales_price = sfo_data["sale_price_sqr_foot"].groupby(["year"]).mean()
    average_sales_price = pd.DataFrame(average_sales_price) 
    average_sales_price = average_sales_price.sale_price_sqr_foot.hvplot.line(
        height = 300, 
        width = 400,
        xlabel = "Year", 
        line_color = "Purple", 
        line_width = 2, 
        ylabel = "Price per SqFt", 
        ylim = (300, 700), 
        title = "Average Price per SqFt by Year")
    return average_sales_price
 

def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    average_price_by_neighborhood = sfo_data.groupby(["year", "neighborhood"]).mean()
    average_price_by_neighborhood = average_price_by_neighborhood.sale_price_sqr_foot.hvplot.line(
        height = 300,
        width = 400,
        xlabel = "Year", 
        line_color = "Purple", 
        line_width = 2, 
        ylabel = "Price per SqFt", 
        ylim = (100, 1000), 
        title = "Average Price per SqFt by Year",
        groupby = 'neighborhood')
    return average_price_by_neighborhood


def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""

    top_most_expensive_neighborhoods = sfo_data.groupby(by="neighborhood").mean()
    top_most_expensive_neighborhoods = top_most_expensive_neighborhoods.nlargest(10,"sale_price_sqr_foot")
    top_most_expensive_neighborhoods.reset_index(inplace=True)
    top_most_expensive_neighborhoods = top_most_expensive_neighborhoods.hvplot.bar(
    height = 400, 
    width = 700, 
    x = "neighborhood", 
    y = "sale_price_sqr_foot",  
    color = "Blue", 
    xlabel = "Neighborhood", 
    ylabel = "Price per Square Foot" , 
    rot = 90, 
    title = "Top Ten Most Expensive Neighborhoods by Price per SqFt")

    return top_most_expensive_neighborhoods
    


def most_expensive_neighborhoods_rent_sales():
    """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""   
    
    avg_df = sfo_data.groupby(["year", "neighborhood"]).mean()
    most_expensive_neighborhoods_rent_sales = avg_df.groupby(by=["year","neighborhood"]).mean()
    most_expensive_neighborhoods_rent_sales.reset_index(inplace=True)
    most_expensive_neighborhoods_rent_sales.sort_values(by="sale_price_sqr_foot", ascending = False)
    most_expensive_neighborhoods_rent_sales = most_expensive_neighborhoods_rent_sales.hvplot.bar(
    height = 400,
    width = 700,
    x = "year",
    y = ["gross_rent", "sale_price_sqr_foot"],
    color = ["Blue","Orange"],
    xlabel = "Neighborhood",
    ylabel = "Housing Units",  
    rot = 90,
    title = "Top Ten Most Expensive Neighborhoods",
    ylim = (100, 4500),
    groupby="neighborhood"
)
    return most_expensive_neighborhoods_rent_sales
    
    
def parallel_coordinates():
    """Parallel Coordinates Plot."""

    top_10_price_per_sqft_by_year = sfo_data.groupby(["neighborhood"]).mean()
    top_10_price_per_sqft_by_year = top_10_price_per_sqft_by_year.nlargest(10, "sale_price_sqr_foot")
    top_10_price_per_sqft_by_year.reset_index(inplace=True)
    parallel_coordinates = px.parallel_coordinates(top_10_price_per_sqft_by_year, 
                                                   color='sale_price_sqr_foot')
    return parallel_coordinates


def parallel_categories():
    """Parallel Categories Plot."""
    
    top_10_price_per_sqft_by_year = sfo_data.groupby(["neighborhood"]).mean()
    top_10_price_per_sqft_by_year = top_10_price_per_sqft_by_year.nlargest(10, "sale_price_sqr_foot")
    top_10_price_per_sqft_by_year.reset_index(inplace=True)
    parallel_categories = px.parallel_categories(
    top_10_price_per_sqft_by_year,
    dimensions=["neighborhood", "sale_price_sqr_foot", "housing_units", "gross_rent"],
    color="sale_price_sqr_foot")
    return parallel_categories


def neighborhood_map():
    """Neighborhood Map."""

    coordinate_data = pd.read_csv(file_path_2)
    avg_values_neighborhood = sfo_data.groupby(by="neighborhood").mean()
    avg_values_neighborhood.reset_index(inplace=True)
    combined_df = pd.concat([avg_values_neighborhood, coordinate_data],axis='columns', join='inner')
    combined_df = combined_df.drop(columns = ["neighborhood"])
    combined_df = combined_df.set_index(["Neighborhood"])
    combined_df.reset_index(inplace=True)
    neighborhood_map = px.scatter_mapbox(
    combined_df,
    lat="Lat",
    lon="Lon",
    size="gross_rent",
    color="gross_rent",
    zoom=10,
    color_continuous_scale = 'blackbody'
)

    return neighborhood_map

## Panel Dashboard

In [6]:
# Create dashboard

yearly_averages = pn.Column(
    "## Yearly Averages", housing_units_per_year(), average_gross_rent(), average_sales_price(), average_price_by_neighborhood()
)

most_expensive = pn.Column(
    "## Top 10 Most Expensive",top_most_expensive_neighborhoods(),most_expensive_neighborhoods_rent_sales()
)

parallel = pn.Column(
    "## Parallel Categories and Coordinates", parallel_categories(), parallel_coordinates()
)

Map = pn.Column(
    "Neighborhood Map", neighborhood_map()
)

# Create tabs
san_fran_RE = pn.Tabs(
    ("Yearly Averages", yearly_averages), ("Most Expensive", most_expensive), ("Parallels", parallel), ("Neighborhood Map", Map)
)

## Serve the Panel Dashboard

In [7]:
# Serve the dashboard

san_fran_RE.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] HoloViews(Bars)
        [2] HoloViews(Curve)
        [3] HoloViews(Curve)
        [4] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
    [1] Column
        [0] Markdown(str)
        [1] HoloViews(Bars)
        [2] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
    [2] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
        [2] Plotly(Figure)
    [3] Column
        [0] Markdown(str)
        [1] Plotly(Figure)